In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'nfile' is the DataFrame from the combined .txt files
# Example data structure
# nfile = pd.read_csv("combined_output.csv")

# Define intervals for 'Peak Field on Sample [mT]'
intervals = [
    {"center": 20, "tolerance": 3},
    {"center": 30, "tolerance": 3},
    {"center": 40, "tolerance": 5},
    {"center": 50, "tolerance": 5}
]

results = []

# Calculate mean and standard deviation for each interval
for interval in intervals:
    lower_bound = interval["center"] - interval["tolerance"]
    upper_bound = interval["center"] + interval["tolerance"]
    
    # Filter data within the interval
    filtered_data = nfile[
        (nfile["Peak Field on Sample [mT]"] >= lower_bound) & 
        (nfile["Peak Field on Sample [mT]"] <= upper_bound)
    ]
    
    # Calculate mean and standard deviation
    mean_resistance = filtered_data["Surface Resistance [nOhm]"].mean()
    std_resistance = filtered_data["Surface Resistance [nOhm]"].std()
    
    # Append results
    results.append({
        "Interval": f"{interval['center']}±{interval['tolerance']}",
        "Mean Resistance (nOhm)": mean_resistance,
        "Std Resistance (nOhm)": std_resistance
    })

# Create a new DataFrame for results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv("resistance_analysis.csv", index=False)

# Print the results
print(results_df)

# Plot the results
plt.errorbar(
    results_df["Interval"],
    results_df["Mean Resistance (nOhm)"],
    yerr=results_df["Std Resistance (nOhm)"],
    fmt='o',
    capsize=5,
    label="Surface Resistance"
)

plt.title("Surface Resistance Analysis")
plt.xlabel("Peak Field Interval [mT]")
plt.ylabel("Surface Resistance [nOhm]")
plt.grid(True)
plt.legend()
plt.savefig("resistance_analysis_plot.pdf", format="pdf", bbox_inches="tight")
plt.show()


In [ ]:
from funclib.importf import *

script_dir = os.path.dirname(os.path.abspath("main.ipynb"))
phd_dir = os.path.dirname(script_dir)
TestPath = os.path.join(phd_dir, "QPR Data", "2022-04-04 - test #36 - ARIES B-3.19 Siegen SIS")

Test1 = HandleTest(TestPath, debug = True)
Test1Data = Test1.load_data(pattern="*41*MHz*.txt")

#pd.set_option('display.max_rows', 100)
Test1.data.info()

#Test1Data.plot.scatter(x="Peak Field on Sample [mT]", y="Surface Resistance [nOhm]")
#plt.show()


In [28]:
from funclib.importf import *

script_dir = os.path.dirname(os.path.abspath("main.ipynb"))
phd_dir = os.path.dirname(script_dir)
TestPath = os.path.join(phd_dir, "QPR Data", "2022-04-04 - test #36 - ARIES B-3.19 Siegen SIS")
print(TestPath)

Test1 = HandleTest(TestPath)
Test1Data = Test1.load_data(pattern="*41*MHz*.txt")

#Test1Data.info()
#pd.set_option('display.max_rows', None)
#print(Test1Data["File Name"])
#print(Test1.Data["Run"].unique())

Test1.plot_histogram(step = 0.5, ParamName=FieldNames.SENS_B, ParamVal=2.5, ParamTol=0.05)


/home/dmitry/other/QPR Data/2022-04-04 - test #36 - ARIES B-3.19 Siegen SIS


([0.0,
  0.5,
  1.0,
  1.5,
  2.0,
  2.5,
  3.0,
  3.5,
  4.0,
  4.5,
  5.0,
  5.5,
  6.0,
  6.5,
  7.0,
  7.5,
  8.0,
  8.5,
  9.0,
  9.5,
  10.0,
  10.5,
  11.0,
  11.5,
  12.0,
  12.5,
  13.0,
  13.5,
  14.0,
  14.5,
  15.0,
  15.5,
  16.0,
  16.5,
  17.0,
  17.5,
  18.0,
  18.5,
  19.0,
  19.5,
  20.0,
  20.5,
  21.0,
  21.5,
  22.0,
  22.5,
  23.0,
  23.5,
  24.0,
  24.5,
  25.0,
  25.5],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  70,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  21,
  0,
  11,
  0,
  20,
  0,
  22,
  0,
  37,
  0,
  30,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  26,
  0],
 510      9.991644
 511      9.991644
 512      9.989960
 513      9.989960
 514      9.989960
           ...    
 1726    19.009018
 1727    19.009018
 1728    19.009156
 1729    19.009156
 1730    19.009156
 Name: Peak Field on Sample [mT], Length: 244, dtype: float64,
 510     107.947341
 511     107.947341
 512     107.926806
 5

In [1]:
import pandas as pd
import numpy as np

def groupe_and_compute(df, x_axis, res=1):
    # Sort and add a row of zeros at the end
    dataset_sort = df.sort_values(by=x_axis, ascending=True).reset_index(drop=True)
    dataset_sort.loc[len(dataset_sort)] = 0

    # note: remove later 
    print("Sorted DataFrame:")
    print(dataset_sort)
    
    index = 0
    avg_list = []
    meanlist = []
    stdlist = []
    
    # Iterate through the dataset
    while index < dataset_sort.index.max():
        current_val = dataset_sort.loc[index, x_axis]
        next_val = dataset_sort.loc[index + 1, x_axis]
        
        if abs(current_val - next_val) <= res:
            avg_list.append(current_val)
        else:
            avg_list.append(current_val)
            meanlist.append(np.mean(avg_list))
            stdlist.append(np.std(avg_list))
            avg_list = []
        index += 1
    
    # Handle the last group if there's anything left
    if avg_list:
        meanlist.append(np.mean(avg_list))
        stdlist.append(np.std(avg_list))
    
    # Create a DataFrame with the results
    result_df = pd.DataFrame({
        x_axis: meanlist,
        x_axis + "_std": stdlist
    })
    
    return result_df


# Create DataFrame
data = {
    "A": [i for i in range(1, 11)]+[12,15,16,45,44,33,34,41,42,43],
    "B": [np.random.choice(range(1, 10), 5, replace=False).tolist() for _ in range(20)]
}

x_axis = "A"
dataset = pd.DataFrame(data)
print("Original DataFrame:")
print(dataset)

sorted_df = groupe_and_compute(dataset,"A",1)

print(sorted_df)


Original DataFrame:
     A                B
0    1  [7, 9, 4, 5, 6]
1    2  [3, 4, 1, 2, 8]
2    3  [2, 9, 4, 7, 6]
3    4  [6, 4, 5, 8, 7]
4    5  [2, 6, 3, 7, 5]
5    6  [9, 6, 2, 4, 5]
6    7  [4, 1, 7, 8, 2]
7    8  [2, 8, 5, 3, 7]
8    9  [4, 6, 1, 7, 8]
9   10  [7, 4, 8, 1, 2]
10  12  [2, 3, 7, 9, 5]
11  15  [4, 1, 3, 2, 5]
12  16  [8, 3, 6, 1, 7]
13  45  [6, 2, 3, 8, 9]
14  44  [5, 8, 6, 7, 9]
15  33  [2, 5, 3, 4, 8]
16  34  [6, 9, 7, 5, 8]
17  41  [8, 4, 1, 3, 5]
18  42  [3, 4, 5, 7, 1]
19  43  [8, 2, 5, 7, 4]
Sorted DataFrame:
     A                B
0    1  [7, 9, 4, 5, 6]
1    2  [3, 4, 1, 2, 8]
2    3  [2, 9, 4, 7, 6]
3    4  [6, 4, 5, 8, 7]
4    5  [2, 6, 3, 7, 5]
5    6  [9, 6, 2, 4, 5]
6    7  [4, 1, 7, 8, 2]
7    8  [2, 8, 5, 3, 7]
8    9  [4, 6, 1, 7, 8]
9   10  [7, 4, 8, 1, 2]
10  12  [2, 3, 7, 9, 5]
11  15  [4, 1, 3, 2, 5]
12  16  [8, 3, 6, 1, 7]
13  33  [2, 5, 3, 4, 8]
14  34  [6, 9, 7, 5, 8]
15  41  [8, 4, 1, 3, 5]
16  42  [3, 4, 5, 7, 1]
17  43  [8, 2, 5, 7, 4]
18

In [3]:
%matplotlib tk
import matplotlib.pyplot as plt

# Original data as unfilled red circles
plt.scatter(dataset["A"], dataset["A"], facecolors='none', edgecolors='red', marker='o', label='Original Data')

# Mean vs Mean with error bars in x, blue square markers, green error bars, and caps
plt.errorbar(sorted_df["A"], sorted_df["A"], 
             xerr=sorted_df["A_std"], 
             marker='s', 
             #markerfacecolor='blue', 
             markeredgecolor='black', 
             ecolor='black', 
             capsize=3,
             linestyle='none',
             label='Mean w/ Error')

plt.xlabel("A")
plt.ylabel("A")
plt.title("Scatter Plots: Original vs Mean & Std")
plt.legend()

fig = plt.gcf()  # Get the current figure
fig.canvas.manager.set_window_title("My Custom Window Title")

plt.show()


In [30]:
import numpy as np
import matplotlib.pyplot as plt
err_or_plt = 'plt'
BFieldVal=10
RunN = 1


PlXax = dataset["A"]
PlYax = dataset["A"]
PlXax2 = sorted_df["A"]
PlYax2= sorted_df["A"]
PlYer2 = sorted_df["A_std"]


# Define colors and palettes
Pal1 = np.array([
    [89, 188, 206],
    [0, 115, 189],
    [217, 83, 25],
    [225, 122, 0],
    [255, 215, 0]
]) / 255

Pal3 = np.array([
    [254, 239, 229],
    [0, 145, 110],
    [0, 115, 189],
    [217, 83, 25],
    [255, 207, 0]
]) / 255

# Choose palette
Pal = Pal3

# Plot settings
sc1 = 1  # Scaling
xlabelN = 'Sample temperature T (K)'
ylabelN = 'Surface resistance Rs (nOhm)'
freq = 416.0*10**6 #str(round(PlotModeData[0]["dataline"][0][1] / 1e6, 0))
plot1name = f"{freq} MHz, B {BFieldVal} mT Run {RunN}"

MarkSize = 7 * sc1
LineW = 1.5 * sc1
FontS = 18 * sc1
MarkShape = 'o'
MarkColor = 'none'
Lcol = Pal[3]

# Create figure
plt.figure(figsize=(9 * sc1, 6 * sc1))

# Plot data

plt.errorbar(PlXax2, PlYax2, xerr= 1 * PlYer2,
                fmt="s",
                label=plot1name,
                linewidth=LineW,
                color=Lcol,
                markeredgecolor="black",
                markerfacecolor=Pal[2],
                capsize=3,
                ecolor='black',
                markersize=6,
                markeredgewidth=1.5)

plt.plot(PlXax, PlYax,
            MarkShape,
            label=plot1name,
            linewidth=LineW,
            color=Lcol,
            markeredgecolor=Lcol,
            markerfacecolor=MarkColor,
            markersize=MarkSize,
            markeredgewidth=1.5)

# Finalize plot
plt.xlabel(xlabelN, fontsize=FontS, fontname='serif')
plt.ylabel(ylabelN, fontsize=FontS, fontname='serif')
plt.grid(False)
plt.box(True)
# Adjust tick parameters
ax = plt.gca()
ax.tick_params(width=1, labelsize=FontS)
for label in ax.get_xticklabels():
    label.set_fontsize(FontS)
    label.set_fontname('serif')
for label in ax.get_yticklabels():
    label.set_fontsize(FontS)
    label.set_fontname('serif')

plt.gca().spines['top'].set_linewidth(1)
plt.gca().spines['right'].set_linewidth(1)
plt.gca().spines['left'].set_linewidth(1)
plt.gca().spines['bottom'].set_linewidth(1)


plt.xlim([0, 50])
plt.tight_layout()
plt.show()
